In [ ]:
import pandas as pd
import numpy as np
from keras.models import load_model
import joblib
from datetime import datetime
import os

def print_model_summary(model):
    """Função para imprimir o resumo e configurações do modelo"""
    print("\n=== Configurações da Rede Neural ===")
    
    # Imprime o resumo padrão do modelo
    print("\nResumo do Modelo:")
    model.summary()
    
    # Obtém e imprime os detalhes de cada camada
    print("\nDetalhes das Camadas:")
    for i, layer in enumerate(model.layers):
        print(f"\nCamada {i+1}: {layer.name}")
        print(f"Tipo: {layer.__class__.__name__}")
        if hasattr(layer, 'units'):
            print(f"Neurônios/Unidades: {layer.units}")
        if hasattr(layer, 'activation'):
            print(f"Função de Ativação: {layer.activation.__name__}")
        if hasattr(layer, 'return_sequences'):
            print(f"Return Sequences: {layer.return_sequences}")
        if hasattr(layer, 'rate'):
            print(f"Dropout Rate: {layer.rate}")
        if hasattr(layer, 'kernel_regularizer') and layer.kernel_regularizer:
            print(f"Regularização: {layer.kernel_regularizer.get_config()}")
    
    # Obtém a configuração do otimizador
    optimizer = model.optimizer
    print("\nConfigurações do Otimizador:")
    print(f"Tipo: {optimizer.__class__.__name__}")
    print(f"Learning Rate: {optimizer.learning_rate.numpy()}")
    
    # Obtém a função de loss
    print("\nFunção de Loss:")
    print(model.loss)

def fazer_previsoes_futuras():
    # Carregar modelo e scaler
    model = load_model('melhor_modelo.h5', custom_objects={'mape_loss': lambda y_true, y_pred: 0})
    scaler = joblib.load('scaler.save')
    
    # Imprimir configurações do modelo
    print_model_summary(model)
    
    # Carregar dados históricos para obter a última janela
    path = 'E:\\Projetos\\ABMS-WP'
    caminho_arquivo = os.path.join(path, 'includes\\Tabela_consumo_Itapua_120m.csv')
    df = pd.read_csv(caminho_arquivo, sep=';')
    df['AM_REFERENCIA'] = pd.to_datetime(df['AM_REFERENCIA'], format='%Y%m')
    df_aggregated = df.groupby('AM_REFERENCIA')['HCLQTCON'].sum().reset_index()
    df_aggregated = df_aggregated.sort_values(by='AM_REFERENCIA')
    
    # Normalizar dados históricos
    ts_scaled = scaler.transform(df_aggregated[['HCLQTCON']]).flatten()
    
    # Preparar a última janela de dados para previsão
    look_back = 6
    last_window = ts_scaled[-look_back:]
    
    # Gerar datas futuras (janeiro/2025 a janeiro/2035)
    future_dates = pd.date_range(start='2025-01-01', end='2035-01-01', freq='MS')
    
    # Fazer previsões passo a passo
    future_predictions = []
    current_window = last_window.copy()
    
    for _ in range(len(future_dates)):
        # Preparar entrada para o modelo
        x = current_window.reshape(1, look_back, 1)
        
        # Fazer previsão
        pred = model.predict(x, verbose=0)
        
        # Armazenar previsão
        future_predictions.append(pred[0, 0])
        
        # Atualizar janela (descartar o mais antigo, adicionar a previsão)
        current_window = np.append(current_window[1:], pred[0, 0])
    
    # Desnormalizar as previsões
    future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1)).flatten()
    
    # Criar DataFrame com resultados
    results_df = pd.DataFrame({
        'Data': future_dates,
        'Valor Previsto': future_predictions
    })
    
    # Salvar em CSV
    results_df.to_csv('previsoes_futuras_2025_2035.csv', index=False)
    print("\nPrevisões salvas em 'previsoes_futuras_2025_2035.csv'")
    
    return results_df

resultados = fazer_previsoes_futuras()
print("\nPrevisões para Janeiro/2025 a Janeiro/2035:")
print(resultados)


=== Configurações da Rede Neural ===

Resumo do Modelo:
Model: "sequential_60"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_89 (LSTM)              (None, 64)                16896     
                                                                 
 dropout_89 (Dropout)        (None, 64)                0         
                                                                 
 dense_118 (Dense)           (None, 25)                1625      
                                                                 
 dense_119 (Dense)           (None, 1)                 26        
                                                                 
Total params: 18547 (72.45 KB)
Trainable params: 18547 (72.45 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________

Detalhes das Camadas:

Camada 1: lstm_89
Tipo: LSTM
Neurônios/Unidades: 64
Função de Ativaç